In [2]:
import math
import pandas as pd
import random
import numpy as np
import pickle
import os.path
from scipy.stats import entropy
from itertools import permutations


def safe_log2(x):
    return math.log2(x) if x > 0 else 0


def tuple_to_string(num_tuple):
    return ''.join(map(str, num_tuple))


def calculate_bulls_cows(source, target):
    if len(source) != len(target):
        raise ValueError("Input arrays must have the same length")

    bulls = sum(s == t for s, t in zip(source, target))
    common_digits = set(source) & set(target)
    cows = sum(min(source.count(digit), target.count(digit)) for digit in common_digits) - bulls

    return bulls, cows


def parse_bulls_n_cows_map_name(digits, guesses={}):
	suffix = ''.join(f'_{k}:{v[0]}{v[1]}' for (k, v) in sorted(guesses.items()))
	return f'bulls_n_cows_map/{digits}{suffix}.pkl'


def initialize(originals, digits):
    filepath = parse_bulls_n_cows_map_name(digits=digits)
    if os.path.isfile(filepath):
        return

    bulls_n_cows_map = {}
    for i, _ in enumerate(originals):
        bulls_n_cows_map[i] = {}
        for di in range(digits+1):
            for dj in range(di+1):
                bulls_n_cows_map[i][(dj, di-dj)] = set()

    for i, source in enumerate(originals):
        bulls_n_cows_map[i][(digits, 0)].add(i)
        for j in range(i):
            target = originals[j]
            bulls_n_cows = calculate_bulls_cows(source, target)
            bulls_n_cows_map[i][bulls_n_cows].add(j)
            bulls_n_cows_map[j][bulls_n_cows].add(i)

    with open(filepath, 'wb') as f:
    	pickle.dump(bulls_n_cows_map, f, protocol=pickle.HIGHEST_PROTOCOL)


def convert_bulls_n_cows_map(originals, bulls_n_cows_map):
	return {originals[i]: {bc: set(originals[j] for j in bulls_n_cows_map[i][bc]) for bc in bulls_n_cows_map[i] if len(bulls_n_cows_map[i][bc]) > 0} for i in bulls_n_cows_map}


def read_bulls_n_cows_map(digits, curr_guesses={}):
	filepath = parse_bulls_n_cows_map_name(digits, curr_guesses)
	if not os.path.isfile(filepath):
		return None

	with open(filepath, 'rb') as f:
		bulls_n_cows_map = pickle.load(f)

	return bulls_n_cows_map


def update_bulls_n_cows_map(org_idx_map, guess, bulls_n_cows, digits, curr_guesses={}):
    next_guesses = curr_guesses.copy()
    next_guesses[guess] = bulls_n_cows

    filepath = parse_bulls_n_cows_map_name(digits=digits, guesses=next_guesses)
    if os.path.isfile(filepath):
        return read_bulls_n_cows_map(digits=digits, curr_guesses=next_guesses)

    bulls_n_cows_map = read_bulls_n_cows_map(digits=digits, curr_guesses=curr_guesses)

    guess_idx = org_idx_map[guess]	
    candidates = bulls_n_cows_map[guess_idx][bulls_n_cows]

    for src_idx in bulls_n_cows_map:
        for bc in bulls_n_cows_map[src_idx]:
            bulls_n_cows_map[src_idx][bc] = bulls_n_cows_map[src_idx][bc].intersection(candidates)

    with open(filepath, 'wb') as f:
        pickle.dump(bulls_n_cows_map, f, protocol=pickle.HIGHEST_PROTOCOL)
        
    return bulls_n_cows_map


def calc_candidates(bulls_n_cows_map):
    candidates = set()
    for src_idx in bulls_n_cows_map:
        for bc in bulls_n_cows_map[src_idx]:
            candidates = candidates.union(bulls_n_cows_map[src_idx][bc])

    return candidates


def calc_entropy_score_map(bulls_n_cows_map, candidates, candidate_entropy, guess_count):
    entropy_map = {}
    score_map = {}
    
    def calc_score(entropy):
        return np.sqrt(entropy)

    C = len(candidates)
    for idx in bulls_n_cows_map:
        factor = (1 - 1/C) if idx in candidates else 1
        entropy_map[idx] = entropy([len(bulls_n_cows_map[idx][bc]) for bc in bulls_n_cows_map[idx]], base=2)
        score_map[idx] = guess_count + calc_score(candidate_entropy - entropy_map[idx]) * factor

    return entropy_map, score_map


def guess_based_on_candidates(originals, org_idx_map, digits, guesses, guess, bulls_n_cows, candidate_entropy, verbose=False):
    bulls_n_cows_map = update_bulls_n_cows_map(org_idx_map=org_idx_map, digits=digits, curr_guesses=guesses, guess=guess, bulls_n_cows=bulls_n_cows)

    guesses[guess] = bulls_n_cows
    candidates = calc_candidates(bulls_n_cows_map=bulls_n_cows_map)

    if verbose:
        print("\n💬", guesses)
        print("🎯", sorted([originals[c] for c in candidates]))

    if len(candidates) == 0:
        return 0, None, None
    elif len(candidates) == 1:
        return 1, originals[list(candidates)[0]], None

    entropy_map, score_map = calc_entropy_score_map(bulls_n_cows_map=bulls_n_cows_map, candidates=candidates, candidate_entropy=candidate_entropy, guess_count=len(guesses))
    best_guess = random.choice(list(candidates))
    
    if verbose:
        candidate_map = {}
        for idx in entropy_map:
            entropy = f'{entropy_map[idx]:.2f}B'
            if entropy in candidate_map:
                candidate_map[entropy].add(originals[idx])
            else:
                 candidate_map[entropy] = set([originals[idx]])
        
        print(f"🎲 {safe_log2(len(candidates)):.2f}B - {entropy_map[best_guess]:.2f}B | {score_map[best_guess]:.2f}P ('{originals[best_guess]}')")
        print(sorted(candidate_map.items(), key = lambda item: item[0]))

    return len(candidates), originals[best_guess], entropy_map[best_guess]


def guess_based_on_entropy(originals, org_idx_map, digits, guesses, guess, bulls_n_cows, candidate_entropy, verbose=False):
    bulls_n_cows_map = update_bulls_n_cows_map(org_idx_map=org_idx_map, digits=digits, curr_guesses=guesses, guess=guess, bulls_n_cows=bulls_n_cows)

    guesses[guess] = bulls_n_cows
    candidates = calc_candidates(bulls_n_cows_map=bulls_n_cows_map)

    if verbose:
        print("\n💬", guesses)
        print("🎯", sorted([originals[c] for c in candidates]))

    if len(candidates) == 0:
        return 0, None, None
    elif len(candidates) == 1:
        return 1, originals[list(candidates)[0]], None

    entropy_map, score_map = calc_entropy_score_map(bulls_n_cows_map=bulls_n_cows_map, candidates=candidates, candidate_entropy=candidate_entropy, guess_count=len(guesses))
    best_guess = max(entropy_map, key=entropy_map.get)
    
    if verbose:
        candidate_map = {}
        for idx in entropy_map:
            entropy = f'{entropy_map[idx]:.2f}B'
            if entropy in candidate_map:
                candidate_map[entropy].add(originals[idx])
            else:
                 candidate_map[entropy] = set([originals[idx]])
        
        print(f"🎲 {safe_log2(len(candidates)):.2f}B - {entropy_map[best_guess]:.2f}B | {score_map[best_guess]:.2f}P ('{originals[best_guess]}')")
        print(sorted(candidate_map.items(), key = lambda item: item[0]))

    return len(candidates), originals[best_guess], entropy_map[best_guess]


def guess_based_on_score(originals, org_idx_map, digits, guesses, guess, bulls_n_cows, candidate_entropy, verbose=False):
    bulls_n_cows_map = update_bulls_n_cows_map(org_idx_map=org_idx_map, digits=digits, curr_guesses=guesses, guess=guess, bulls_n_cows=bulls_n_cows)

    guesses[guess] = bulls_n_cows
    candidates = calc_candidates(bulls_n_cows_map=bulls_n_cows_map)

    if verbose:
        print("\n💬", guesses)
        print("🎯", sorted([originals[c] for c in candidates]))

    if len(candidates) == 0:
        return 0, None, None
    elif len(candidates) == 1:
        return 1, originals[list(candidates)[0]], None

    entropy_map, score_map = calc_entropy_score_map(bulls_n_cows_map=bulls_n_cows_map, candidates=candidates, candidate_entropy=candidate_entropy, guess_count=len(guesses))
    best_guess = min(score_map, key=score_map.get)

    if verbose:
        candidate_map = {}
        for idx in score_map:
            score = f'{score_map[idx]:.2f}P'
            if score in candidate_map:
                candidate_map[score].add(originals[idx])
            else:
                 candidate_map[score] = set([originals[idx]])

        print(f"🎲 {safe_log2(len(candidates)):.2f}B - {entropy_map[best_guess]:.2f}B | {score_map[best_guess]:.2f}P ('{originals[best_guess]}')")
        print(sorted(candidate_map.items(), key = lambda item: item[0]))

    return len(candidates), originals[best_guess], entropy_map[best_guess]


In [3]:
class BullsNCows:
    def __init__(self, digits=4, guess_algorithm=guess_based_on_score, verbose=False):
        permute = permutations([i for i in range(10)], digits)
        self.originals = [tuple_to_string(p) for p in list(permute)]
        self.org_idx_map = {org: idx for idx, org in enumerate(self.originals)}
        self.digits = digits
        self.guess = guess_algorithm
        self.verbose = verbose
        self.reset()

        initialize(originals=self.originals, digits=4)


    def reset(self):
        self.guesses = {}
        self.secret = random.choice(self.originals)
        self.summary = [{
            "guess": "",
            "guess_result": "",
            "candidate_count": len(self.originals),
            "candidate_entropy": safe_log2(len(self.originals)),
            "guess_actual_entropy": np.NaN,
            "best_guess_entropy": np.NaN,
            "best_guess": "",
        }]


    def next(self):
        if len(self.guesses) > 0:
            guess = self.summary[-1]['best_guess']
        else:
            guess = random.choice(self.originals)
            idx = self.org_idx_map[guess]
            bulls_n_cows_map = read_bulls_n_cows_map(digits=self.digits)

            self.summary[-1]['best_guess'] = guess
            self.summary[-1]['best_guess_entropy'] = entropy([len(bulls_n_cows_map[idx][bc]) for bc in bulls_n_cows_map[idx]], base=2)
 
        guess_result = calculate_bulls_cows(self.secret, guess)
        candidate_count, best_guess, best_guess_entropy = self.guess(originals=self.originals, org_idx_map=self.org_idx_map, guesses=self.guesses, guess=guess, bulls_n_cows=guess_result, digits=self.digits, candidate_entropy=self.summary[-1]['candidate_entropy'], verbose=self.verbose)

        self.summary.append({
            "guess": guess,
            "guess_result": guess_result,
            "candidate_count": candidate_count,
            "candidate_entropy": safe_log2(candidate_count),
            "guess_actual_entropy": self.summary[-1]['candidate_entropy']-safe_log2(candidate_count),
            "best_guess_entropy": best_guess_entropy,
            "best_guess": best_guess,
        })

        return self


    def play(self, n_iter=10):
        self.reset()
        print(self.secret)
        while self.summary[-1]['guess'] != self.secret and n_iter > 0:
            self.next()
            n_iter -= 1

        return pd.DataFrame.from_dict(self.summary)

In [4]:
game = BullsNCows(digits=2)
C = len(game.originals)
C, _, _ = guess_based_on_score(originals=game.originals, org_idx_map=game.org_idx_map, guesses=game.guesses, guess='54', bulls_n_cows=(1, 0), digits=2, candidate_entropy=safe_log2(C), verbose=True)
C, _, _ = guess_based_on_score(originals=game.originals, org_idx_map=game.org_idx_map, guesses=game.guesses, guess='12', bulls_n_cows=(0, 1), digits=2, candidate_entropy=safe_log2(C), verbose=True)
C, _, _ = guess_based_on_score(originals=game.originals, org_idx_map=game.org_idx_map, guesses=game.guesses, guess='24', bulls_n_cows=(2, 0), digits=2, candidate_entropy=safe_log2(C), verbose=True)


💬 {'54': (1, 0)}
🎯 ['04', '14', '24', '34', '50', '51', '52', '53', '56', '57', '58', '59', '64', '74', '84', '94']
🎲 4.00B - 1.54B | 3.09P ('04')
[('3.09P', {'74', '52', '24', '58', '59', '14', '34', '04', '94', '56', '84', '53', '51', '64', '50', '57'}), ('3.22P', {'85', '46', '15', '47', '65', '40', '41', '48', '75', '25', '42', '43', '95', '35', '49', '05'}), ('3.33P', {'12', '39', '83', '36', '08', '26', '17', '81', '27', '30', '07', '32', '86', '93', '02', '79', '18', '80', '90', '19', '69', '73', '29', '06', '68', '09', '03', '37', '67', '01', '92', '28', '20', '91', '97', '13', '38', '98', '76', '72', '23', '78', '16', '63', '71', '87', '31', '70', '60', '89', '82', '10', '21', '61', '96', '62'}), ('3.55P', {'45', '54'})]

💬 {'54': (1, 0), '12': (0, 1)}
🎯 ['24', '51']
🎲 1.00B - 1.00B | 2.87P ('24')
[('2.87P', {'24', '51'}), ('3.73P', {'52', '47', '81', '26', '17', '40', '41', '95', '27', '35', '56', '94', '05', '32', '74', '02', '85', '15', '18', '59', '19', '25', '43', '29', 

In [5]:
game = BullsNCows(digits=2)
C = len(game.originals)
C, _, _ = guess_based_on_entropy(originals=game.originals, org_idx_map=game.org_idx_map, guesses=game.guesses, guess='54', bulls_n_cows=(1, 0), digits=2, candidate_entropy=safe_log2(C), verbose=True)
C, _, _ = guess_based_on_entropy(originals=game.originals, org_idx_map=game.org_idx_map, guesses=game.guesses, guess='12', bulls_n_cows=(0, 1), digits=2, candidate_entropy=safe_log2(C), verbose=True)
C, _, _ = guess_based_on_entropy(originals=game.originals, org_idx_map=game.org_idx_map, guesses=game.guesses, guess='24', bulls_n_cows=(2, 0), digits=2, candidate_entropy=safe_log2(C), verbose=True)


💬 {'54': (1, 0)}
🎯 ['04', '14', '24', '34', '50', '51', '52', '53', '56', '57', '58', '59', '64', '74', '84', '94']
🎲 4.00B - 1.54B | 3.09P ('04')
[('0.00B', {'45', '54'}), ('1.06B', {'12', '39', '83', '36', '08', '26', '17', '81', '27', '30', '07', '32', '86', '93', '02', '79', '18', '80', '90', '19', '69', '73', '29', '06', '68', '09', '03', '37', '67', '01', '92', '28', '20', '91', '97', '13', '38', '98', '76', '72', '23', '78', '16', '63', '71', '87', '31', '70', '60', '89', '82', '10', '21', '61', '96', '62'}), ('1.54B', {'52', '47', '40', '41', '95', '94', '35', '56', '51', '05', '74', '85', '15', '59', '25', '43', '84', '65', '04', '53', '50', '46', '24', '58', '14', '34', '48', '75', '42', '49', '64', '57'})]

💬 {'54': (1, 0), '12': (0, 1)}
🎯 ['24', '51']
🎲 1.00B - 1.00B | 3.73P ('01')
[('0.00B', {'12', '39', '54', '83', '36', '08', '30', '07', '86', '93', '79', '80', '90', '69', '73', '06', '68', '09', '03', '37', '67', '45', '97', '38', '98', '76', '78', '63', '87', '70', '6

In [6]:
game = BullsNCows(digits=2)
C = len(game.originals)
C, _, _ = guess_based_on_candidates(originals=game.originals, org_idx_map=game.org_idx_map, guesses=game.guesses, guess='54', bulls_n_cows=(1, 0), digits=2, candidate_entropy=safe_log2(C), verbose=True)
C, _, _ = guess_based_on_candidates(originals=game.originals, org_idx_map=game.org_idx_map, guesses=game.guesses, guess='12', bulls_n_cows=(0, 1), digits=2, candidate_entropy=safe_log2(C), verbose=True)
C, _, _ = guess_based_on_candidates(originals=game.originals, org_idx_map=game.org_idx_map, guesses=game.guesses, guess='24', bulls_n_cows=(2, 0), digits=2, candidate_entropy=safe_log2(C), verbose=True)


💬 {'54': (1, 0)}
🎯 ['04', '14', '24', '34', '50', '51', '52', '53', '56', '57', '58', '59', '64', '74', '84', '94']
🎲 4.00B - 1.54B | 3.09P ('64')
[('0.00B', {'45', '54'}), ('1.06B', {'12', '39', '83', '36', '08', '26', '17', '81', '27', '30', '07', '32', '86', '93', '02', '79', '18', '80', '90', '19', '69', '73', '29', '06', '68', '09', '03', '37', '67', '01', '92', '28', '20', '91', '97', '13', '38', '98', '76', '72', '23', '78', '16', '63', '71', '87', '31', '70', '60', '89', '82', '10', '21', '61', '96', '62'}), ('1.54B', {'52', '47', '40', '41', '95', '94', '35', '56', '51', '05', '74', '85', '15', '59', '25', '43', '84', '65', '04', '53', '50', '46', '24', '58', '14', '34', '48', '75', '42', '49', '64', '57'})]

💬 {'54': (1, 0), '12': (0, 1)}
🎯 ['24', '51']
🎲 1.00B - 1.00B | 2.87P ('51')
[('0.00B', {'12', '39', '54', '83', '36', '08', '30', '07', '86', '93', '79', '80', '90', '69', '73', '06', '68', '09', '03', '37', '67', '45', '97', '38', '98', '76', '78', '63', '87', '70', '6

In [7]:
df = BullsNCows(digits=4, guess_algorithm=guess_based_on_entropy, verbose=False).play(); df

6921


: 

: 